<a href="https://colab.research.google.com/github/benbaz-2/comp551/blob/main/assignment_3_pre_trained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
!pip install medmnist

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=ae8f30d7dfab80cd87cf631bfd19b2787596632530fe4c692b6a206abbe2589f
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import numpy as np
import matplotlib.pyplot as plt
from medmnist import OrganAMNIST

In [ ]:
# Import a pre-trained model
resnet = models.resnet18(pretrained=True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 212MB/s]


In [ ]:
# Freezing the parameters
for param in resnet.parameters():
    param.requires_grad = False

In [ ]:
# Replace fully connected layer at the end by an empty container
num_features = resnet.fc.in_features  # Store the size of the input for future use
resnet.fc = nn.Sequential()

In [ ]:
# Add fully connected layers
# I will first try adding two because OrganAMNIST is a relatively small dataset
num_classes = 11  # Number of classes in OrganAMNIST
resnet.fc = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.ReLU(),
    nn.Dropout(0.5),  # for regularization
    nn.Linear(256, num_classes)
)

In [ ]:
# downloading OrganAMNIST
train_dataset = OrganAMNIST(split='train', download=True)
test_dataset = OrganAMNIST(split='test', download=True)
val_dataset = OrganAMNIST(split='val', download=True)

100%|██████████| 38.2M/38.2M [00:04<00:00, 9.11MB/s]


Using downloaded and verified file: /root/.medmnist/organamnist.npz
Using downloaded and verified file: /root/.medmnist/organamnist.npz


In [ ]:
# Specify the transforms
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert grayscale to RGB
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(mean=[0.5], std=[0.5]),  # Normalize
    ])


In [ ]:
# Apply the transforms
train_dataset.transform = transform
test_dataset.transform = transform
val_dataset.transform = transform

In [ ]:
# Prepare data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
# Define loss and optimization algorithm
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet.fc.parameters(), lr=0.001)


In [ ]:
# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet.to(device)

epochs = 10

for epoch in range(epochs):
    resnet.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        labels = labels.squeeze(1)

        optimizer.zero_grad()
        outputs = resnet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")


Epoch 1, Loss: 0.7847385705068749
Epoch 2, Loss: 0.5487619917167766
Epoch 3, Loss: 0.5175187321695104
Epoch 4, Loss: 0.486123065847583
Epoch 5, Loss: 0.4608013222578165
Epoch 6, Loss: 0.45585608579542325
Epoch 7, Loss: 0.455691572482061
Epoch 8, Loss: 0.4417227110881479
Epoch 9, Loss: 0.4250551891012615
Epoch 10, Loss: 0.43407659923914516


In [ ]:
# Evaluation
resnet.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.squeeze(1)
        outputs = resnet(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Validation Accuracy: {100 * correct / total}%")


Validation Accuracy: 91.66538283777538%


In [ ]:
val_loader.dataset.labels.shape

(6491, 1)

In [ ]:
for images, labels in val_loader:
    print(labels.shape)

torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])


KeyboardInterrupt: 